# Billing Discrepancies for Endurance

- For Endurance, the brand report only shows form referrals. 
- For Calls we bill form our numbers, so I will use the call report from etl_output.referral
- There is no IVR setup in Invoca, so all calls longer than the minimum call duration count towards billing
- UUIDs are in a field called: "Vendor Id2"
- The "Response" field contains the status of the lead. If it is empty, it means the referral is valid. Otherwise, there is an entry that tells us why the lead is rejected


In [139]:
#Import Libaries
#***************
from datetime import datetime  as dt
import pandas as pd
import re
import math

In [163]:
#Print Styles for the report
#***************************

def print_title(text, title_type, spaces_after, spaces_before = 0):

    x = spaces_before
    while x > 0:
        print()
        x -= 1
    
    if title_type == 'h1':
        print(text.upper())
        print('*' * len(text))
    elif title_type == 'h2':
        print(text)
        print('=' * len(text))
    elif title_type == 'h3':
        print(text)
        print('-' * len(text))
    else:
        print(text)
    
    x = spaces_after
    while x > 0:
        print()
        x -= 1
    else:
        return

    
#Transformation functions
#------------------------

#The clean_US_phone_number function makes the following assumptions
#All numbers belong to the US. We only remove the first character of the phone if it is the US country code: 1
#Number can contain extension numbers and other additional info at the end of the number, and not at the beginning
#This function has a lot of deffensive code
def clean_US_phone_number(text_phone_number):
    
    if pd.isnull(text_phone_number):
        return ''
    
    modified = re.sub("\D+","", str(text_phone_number))
    
    if modified == '':
        return ''

    #Remove the country code
    if modified[0] == '1':
        modified = modified[1:]
    
    #Cover corner case: too few characters
    if len(modified) < 10:
        return modified
    
    #Remove anything else at the end of the number
    modified = modified[:10]
    return modified
    
#The following function is more forgiving in the country code but is unable to remove extension info
def clean_phone_number_no_ext(text_phone_number):
    modified = re.sub("\D+","", str(text_phone_number))
    #Remove the country code
    modified = modified[-10:]
    return modified

def clean_phone_duration(phone_duration):
    if math.isnan(phone_duration):
        return 0
    return int(phone_duration)

def is_blank(myString):
    return not (str(myString) and str(myString).strip())

def is_not_blank(myString):
    return bool(str(myString) and str(myString).strip())

#Valid for Pandas DataFrames
def is_blank_or_nan(myString):
    return not (str(myString) and str(myString).strip() and not pd.isna(myString))

#Valid for Pandas DataFrames
def is_not_blank_or_nan(myString):
    return bool(str(myString) and str(myString).strip() and not pd.isna(myString))


In [141]:
#Named variables Definition
#**************************
call_rate = 125
form_rate = 125
minimum_call_duration = 60
#reported_discrepancy = 
actually_billed = 
brand_report_file = "endurance/endurance_brand_report_201908.csv"
ca_report_file = "endurance/endurance_ca_report_201908.csv"
brand_report_separator = ";"

#Field names in Brand Report
brand_report_phone_field_name = 'Phone'
brand_report_uuid_field_name = 'Vendor Id2'
brand_report_email_field_name = 'Email'

#The following are field names for the CA report
#These are not expected to change but I don't want to have to 
#make a massive mod if they do for whatever reason
ca_report_originating_number_field_name = 'originating_number'
ca_report_phone_number_field_name = 'phone_number'
ca_report_phone_form_field_name = 'form_phone'
ca_report_uuid_field_name = 'uuid'
ca_report_email_field_name = 'form_email'
ca_report_referral_path_field_name = 'referral_path'
ca_report_form_referral_path_field_value = 'form'
ca_report_call_referral_path_field_value = 'call'
ca_report_click_referral_path_field_value = 'click'
ca_report_raw_referral_id_field_name = 'raw_referral_id'
ca_report_reason_why_not_billed_field_name = 'reason_why_not_billed'
ca_report_phone_duration_field_name = 'phone_duration'

#------------

SyntaxError: invalid syntax (<ipython-input-141-260bc0518590>, line 6)

In [162]:
#Data Gathering and Wrangling
#****************************

#Load the files. The brand report comes from an Excel file, whose default separator to export to CSV is ;
#he CA report always comes from a Mode query
brand_report = pd.read_csv(brand_report_file, sep = brand_report_separator)
ca_report = pd.read_csv(ca_report_file)

#Data transformation
#-------------------
#First, we want to give the phone numbers a unified format

#Brand
brand_report[brand_report_phone_field_name] = (brand_report[brand_report_phone_field_name].
                                               apply(clean_US_phone_number))
#CA
ca_report[ca_report_originating_number_field_name] = (
    ca_report[ca_report_originating_number_field_name].apply(clean_US_phone_number))
ca_report[ca_report_phone_number_field_name] = (
    ca_report[ca_report_phone_number_field_name].apply(clean_US_phone_number))
ca_report[ca_report_phone_form_field_name] = (
    ca_report[ca_report_phone_form_field_name].apply(clean_US_phone_number))


#Now we make sure the UUIDs and emails are strings!
#Brand
brand_report[brand_report_uuid_field_name] = brand_report[brand_report_uuid_field_name].apply(lambda x: str(x))
brand_report[brand_report_email_field_name] = brand_report[brand_report_email_field_name].apply(lambda x: str(x))
#CA
ca_report[ca_report_uuid_field_name] = ca_report[ca_report_uuid_field_name].apply(lambda x: str(x))
ca_report[ca_report_email_field_name] = ca_report[ca_report_email_field_name].apply(lambda x: str(x))

#Transform phone_duration to integer
#Brand
#ToDo: Add Phone transformation for brand reports that have phone calls

#CA
ca_report[ca_report_phone_duration_field_name] = (
ca_report[ca_report_phone_duration_field_name].apply(clean_phone_duration))



#Add a True|False field to filter rejected referrals easily
#This field will be inherited by all reports: call, form and click
#Note that we do this only on CA side. We do want to see matched referrals
#that are good for CA but bad for the brand 
ca_report['accepted_referral'] = ca_report[ca_report_reason_why_not_billed_field_name].apply(is_blank_or_nan)

#ToDo: Create Call and Form reports for the Brand
#In this case, we only have the form report
brand_report_forms = brand_report.copy()

#Filter Forms, Calls and Clicks from CA report
is_form = ca_report[ca_report_referral_path_field_name] == ca_report_form_referral_path_field_value
is_call = ca_report[ca_report_referral_path_field_name] == ca_report_call_referral_path_field_value
is_click = ca_report[ca_report_referral_path_field_name] == ca_report_click_referral_path_field_value
is_empty = ca_report[ca_report_referral_path_field_name] == ''

#In the following 4 lines, I added the copy() explicitly to avoid working on views
#If the copy() is omitted, the lines where I use a drop_duplicates() statement 
#for CA data raise a settingwithcopywarning
#because of a chain indexing. The following excellent article explains why: 
#https://www.dataquest.io/blog/settingwithcopywarning/
ca_report_forms = ca_report[is_form].copy()
ca_report_calls = ca_report[is_call].copy()
ca_report_clicks = ca_report[is_click].copy()
ca_report_no_ref_path = ca_report[is_empty].copy()

#Count duplicates in each referral_path
#First we count the numbers including duplicates
#Brand report: For endurance they only report forms
brand_form_leads_number = len(brand_report)
ca_form_referrals_number = len(ca_report_forms)
ca_call_referrals_number = len(ca_report_calls)
ca_click_referrals_number = len(ca_report_clicks)
ca_no_ref_path_referrals_number = len(ca_report_no_ref_path)

#Drop Duplicates and count again
#-------------------------------

#Unique identifier for the brand form referrals: uuid sent in Vendor Id2 field
brand_report.drop_duplicates(subset = brand_report_uuid_field_name, keep='first', inplace=True)
brand_form_leads_dedup_number = len(brand_report)

#Unique identifier for CA form referrals: uuid
ca_report_forms.drop_duplicates(subset = ca_report_uuid_field_name, keep='first', inplace=True)
ca_form_referrals_dedup_number = len(ca_report_forms)

#Unique identifier for CA call referrals: originating_number
ca_report_calls.drop_duplicates(subset = ca_report_originating_number_field_name, keep='first', inplace=True)
ca_call_referrals_dedup_number = len(ca_report_calls)

#Unique identifier for CA click referrals: uuid
ca_report_clicks.drop_duplicates(subset = ca_report_uuid_field_name, keep='first', inplace=True)
ca_click_referrals_dedup_number = len(ca_report_clicks)



#Show results
#-------------
print_title('data gathering and wrangling section', 'h1', 2)
print_title('Files Analysis','h2', 1)

#Brand Report
print_title('Brand General Report','h3', 1)
brand_form_leads_duplicate_number = brand_form_leads_number - brand_form_leads_dedup_number
print("There are {0} form referrals in the brand report".format(brand_form_leads_number))
print("There are {0} duplicate form referrals in the brand report".format(brand_form_leads_duplicate_number))
print_title('Shape and List of Fields','h3', 1, 1)
print("This is the shape of the Brand DataFrame: {0}".format(brand_report.shape))
print_title('Brand report DataTypes', 'h3', 1, 1)
print(brand_report.dtypes)
print(brand_report.head(10))
print_title('Bad Leads Classification', 'h3', 1, 1)
brand_rejected_lead_types = brand_report[['Id','Response']].groupby('Response').count()
print(brand_rejected_lead_types)
#The ones with empty 'Response' value are not selected in the final brand_lead_types
#Those with empty values are accepted leads according to the brand report
brand_rejected_lead_types_number = brand_lead_types['Id'].sum()
brand_accepted_lead_types_number = len(brand_report)-brand_rejected_lead_types_number
print("There are {0} bad leands and {1} good leads in the brand report"
      .format(brand_rejected_lead_types_number, brand_accepted_lead_types_number))


#CA Report
print_title('CA General Report','h3', 1, 1)
ca_form_referrals_duplicate_number = ca_form_referrals_number - ca_form_referrals_dedup_number
print('There are {0} unique form referrals in the CA report'.format(ca_form_referrals_dedup_number))
print("There are {0} duplicate form referrals in the brand report".format(ca_form_referrals_duplicate_number))
ca_call_referrals_duplicate_number = ca_call_referrals_number - ca_call_referrals_dedup_number
print("There are {0} unique call referrals in the CA report".format(ca_call_referrals_dedup_number))
print("There are {0} duplicate call referrals in the CA report".format(ca_call_referrals_duplicate_number))
ca_click_referrals_duplicate_number = ca_click_referrals_number - ca_click_referrals_dedup_number
print("There are {0} unique click referrals in the CA report".format(ca_click_referrals_dedup_number))
print("There are {0} duplicate click referrals in the CA report".format(ca_click_referrals_duplicate_number))

print_title('Shape and List of Fields','h3', 1, 1)
print("This is the shape of the CA DataFrame: {0}".format(ca_report.shape))
print_title('CA report DataTypes', 'h3', 1, 1)
print(ca_report.dtypes)

#In the following we only count Form and Call referrals as usually click referrals are non-billable
print_title('Non-Billable CA Form Referrals Classification', 'h3', 1, 1)
ca_non_billable_form_referral_types = (
    ca_report_forms[[ca_report_raw_referral_id_field_name, ca_report_reason_why_not_billed_field_name]].
                                       groupby(ca_report_reason_why_not_billed_field_name).count())
print(ca_non_billable_form_referral_types)

ca_non_billable_form_referrals_number = (ca_non_billable_form_referral_types[
    ca_report_raw_referral_id_field_name].sum())
print("There are {0} non-billable Form referrals and {1} billable Form referrals in the CA report".
      format(ca_non_billable_form_referrals_number, len(ca_report_forms)-ca_non_billable_form_referrals_number))

#Analize Billable form referrals based on reason_why_not_billed field
ca_report_billable_forms = ca_report_forms.loc[(ca_report_forms['accepted_referral'])]
print('Created a Billable Forms dataframe with {0} referrals'.format(len(ca_report_billable_forms)))


print_title('Non-Billable CA Call Referrals Classification', 'h3', 1, 1)
ca_non_billable_call_referral_types = (
    ca_report_calls[[ca_report_raw_referral_id_field_name, ca_report_reason_why_not_billed_field_name]].
                                       groupby(ca_report_reason_why_not_billed_field_name).count())
print(ca_non_billable_call_referral_types)
ca_non_billable_call_referrals_number = (
    ca_non_billable_call_referral_types[ca_report_raw_referral_id_field_name].sum())
print("There are {0} non-billable Call referrals and {1} billable Call referrals in the CA report according to Matillion".
      format(ca_non_billable_call_referrals_number, len(ca_report_calls)-ca_non_billable_call_referrals_number))


#Analize billable calls based on duration
is_long_enough_filter = ca_report_calls[ca_report_phone_duration_field_name] >= minimum_call_duration

ca_report_billable_calls = ca_report_calls.loc[(ca_report_calls['accepted_referral']) &
                                              (ca_report_calls[ca_report_phone_duration_field_name] >= minimum_call_duration)]
print('Number of billable calls on the CA report after checking duration locally: {0}'.format(len(ca_report_billable_calls)))


DATA GATHERING AND WRANGLING SECTION
************************************


Files Analysis

Brand General Report
--------------------

There are 21120 form referrals in the brand report
There are 10 duplicate form referrals in the brand report

Shape and List of Fields
------------------------

This is the shape of the Brand DataFrame: (21110, 23)

Brand report DataTypes
----------------------

Report date         object
Id                   int64
Account Number      object
Disposition         object
Last Action         object
Revised Response    object
Status              object
Reason              object
First Name          object
Last Name           object
Phone               object
Email               object
Response            object
Vendor Id            int64
Vendor Id2          object
Year                 int64
Make                object
Model               object
Mileage              int64
City                object
State               object
Website             object
Ip Addre

In [161]:
#Matching Referrals
#We have now cleaned up CA referrals 
#The next step is to match the CA referrals with Brand referrals and see if there are:
#non-matched referrals, or
#matched referrals that can't be billed according to the Brand
#Also, we will use a right join, so we can count how many brand_referrals were not found on CA side
#Before merging, I need to add a field in the brand that has the same name as the one on CA and will
#be used for the merge operation

#Matching Forms
brand_report_forms[ca_report_uuid_field_name] = brand_report_forms[brand_report_uuid_field_name]
right_matched_forms = (ca_report_billable_forms.
                                    merge(brand_report_forms, on = ca_report_uuid_field_name, how = 'right'))

#Now I will classify the ca referrals in three groups: matched and billable, 
#matched and non-billable, and unmatched
#They will be called: ca_report_matched_billable_forms (calls), 
# ca_report_matched_billable_forms (calls), and ca_report_unmatched_forms (calls)

#The brand referrals will be grouped in: Matched and unmatched

#Create a DataFrame with non matched Brand form referrals only
#ca_match_filter = not pd.isna(right_matched_forms[ca_report_raw_referral_id_field_name])
#ca_report_matched_billable_forms = right_matched_forms[ca_match_filter].copy()
ca_report_matched_forms = (right_matched_forms[
    right_matched_forms[ca_report_raw_referral_id_field_name].notnull()])

ca_report_unmatched_forms = (ca_report_forms[(~ca_report_forms[ca_report_raw_referral_id_field_name].
                                              isin(ca_report_matched_forms[ca_report_raw_referral_id_field_name]))])
print("Number of CA Forms that are not matched: {0}".format(len(ca_report_unmatched_forms)))



brand_report_unmatched_billable_forms = (right_matched_forms[
    right_matched_forms[ca_report_raw_referral_id_field_name].isnull()])

print("Number of CA Forms matched: {0}".format(len(ca_report_matched_billable_forms)))

                                    
#ToDo: Matching Calls                                    
                                    
                                    

Number of CA Forms that are not matched: 256
Number of CA Forms matched: 20031
